## 🧪 IF-AHP Evaluation and Validation Notebook

This Jupyter notebook serves as the **evaluation and validation script** to reproduce the process described in the accompanying research paper. The Intuitionistic Fuzzy Preference Relation (IFPR) matrix defined below is taken directly from the **case study section** of the research to demonstrate and verify the correctness of the implemented code.

The structure of the matrix and the computational methodology are based on the formulation proposed in the paper titled:

> **"Intuitionistic Fuzzy Analytic Hierarchy Process"**  
> *Zeshui Xu and Huchang Liao*  
> [ResearchGate Link](https://www.researchgate.net/publication/264387170_Intuitionistic_Fuzzy_Analytic_Hierarchy_Process)

---

The matrix represents expert evaluations of five criteria with respect to a common goal using intuitionistic fuzzy values — each pair given as **(μ, ν)**, where:

- **μ**: Degree of preference  
- **ν**: Degree of non-preference  
- **π**: Hesitation degree, implicitly defined as **π = 1 − μ − ν**

---

Feel free to **modify or replace this matrix** with your own pairwise evaluations if you'd like to analyze different decision-making scenarios using the same IF-AHP methodology.


In [34]:
import numpy as np
import pandas as pd


# RESEARCH PAPER CASE STUDY MATRIX
R = np.array([
    [[0.5, 0.5], [0.6, 0.2], [0.6, 0.2], [0.65, 0.25], [0.65, 0.25]],
    [[0.2, 0.6], [0.5, 0.5], [0.6, 0.2], [0.65, 0.25], [0.65, 0.25]],
    [[0.4, 0.2], [0.4, 0.2], [0.5, 0.5], [0.6, 0.2], [0.6, 0.2]],
    [[0.35, 0.65], [0.35, 0.65], [0.4, 0.2], [0.5, 0.5], [0.6, 0.2]],
    [[0.35, 0.65], [0.35, 0.65], [0.4, 0.2], [0.4, 0.2], [0.5, 0.5]]
])

### 🧮 Constructing the Perfect Multiplicative Consistent IFPR Matrix

This function transforms the original Intuitionistic Fuzzy Preference Relation (IFPR) matrix into a **Perfect Multiplicative Consistent IFPR** (`R̄`). It enforces global consistency in pairwise comparisons by:

- ⚙️ **Setting diagonal elements** to (0.5, 0.5), representing neutral self-comparisons.
- 🔗 **Recomputing non-adjacent upper-triangular values** using multiplicative chaining of intermediate pairs with exponential smoothing — ensuring indirect consistency.
- 🔁 **Filling the lower triangle** via reciprocity, i.e., `R̄[k, i] = (ν, μ)` if `R̄[i, k] = (μ, ν)`.

This transformation is a **preprocessing step required for IF-AHP** to ensure reliable priority vector computation.
 calculations.
 calculations.


In [72]:
def construct_perfect_multiplicative_consistent_ifpr(R):
    n = R.shape[0]
    R_bar = np.zeros((n, n, 2), dtype=float)

    # 1. Set diagonal entries to (0.5, 0.5)
    for i in range(n):
        R_bar[i, i] = [0.5, 0.5]

    # 2. Compute upper-triangular entries
    for i in range(n):
        for k in range(i+1, n):
            if k == i + 1:
                # For adjacent indices, simply use the original value.
                R_bar[i, k] = R[i, k]
            else:
                exponent = 1.0 / (k - i - 1)
                # Compute chain-products for membership
                prod_mu = 1.0
                prod_1mu = 1.0
                for t in range(i+1, k):
                    prod_mu   *= (R[i, t, 0] * R[t, k, 0])
                    prod_1mu  *= ((1 - R[i, t, 0]) * (1 - R[t, k, 0]))
                left_mu  = prod_mu ** exponent
                right_mu = prod_1mu ** exponent
                denom_mu = left_mu + right_mu
                new_mu = left_mu / denom_mu if denom_mu != 0 else 0.5

                # Compute chain-products for non-membership
                prod_nu = 1.0
                prod_1nu = 1.0
                for t in range(i+1, k):
                    prod_nu   *= (R[i, t, 1] * R[t, k, 1])
                    prod_1nu  *= ((1 - R[i, t, 1]) * (1 - R[t, k, 1]))
                left_nu  = prod_nu ** exponent
                right_nu = prod_1nu ** exponent
                denom_nu = left_nu + right_nu
                new_nu = left_nu / denom_nu if denom_nu != 0 else 0.5

                R_bar[i, k] = [new_mu, new_nu]

    # 3. Fill the lower-triangular part using reciprocity:
    # If R_bar[i,k] = (μ, ν), then set R_bar[k,i] = (ν, μ).
    for i in range(n):
        for k in range(i+1, n):
            mu_val, nu_val = R_bar[i, k]
            R_bar[k, i] = [nu_val, mu_val]

    return R_bar

# Construct the Perfect Multiplicative Consistent IFPR
R_bar = construct_perfect_multiplicative_consistent_ifpr(R)

# Print results with better formatting
np.set_printoptions(precision=4, suppress=True)
print("Original R (upper triangle):")
print(R[..., 0], "\n", R[..., 1], "\n")  # Print membership (μ) and non-membership (ν) matrices

print("Perfect Multiplicative Consistent IFPR (R_bar):")

# Dynamically determine the size of the matrix
n = R.shape[0]  # Number of rows (and columns, since it's an N x N matrix)

# Loop through each row and column dynamically
for i in range(n):
    row_str = []
    for k in range(n):
        mu_val = R_bar[i, k, 0]
        nu_val = R_bar[i, k, 1]
        row_str.append(f"({mu_val:.4f}, {nu_val:.4f})")
    print(f"Row {i+1}:", "\t".join(row_str))

Original R (upper triangle):
[[0.5  0.6  0.6  0.65 0.65]
 [0.2  0.5  0.6  0.65 0.65]
 [0.4  0.4  0.5  0.6  0.6 ]
 [0.35 0.35 0.4  0.5  0.6 ]
 [0.35 0.35 0.4  0.4  0.5 ]] 
 [[0.5  0.2  0.2  0.25 0.25]
 [0.6  0.5  0.2  0.25 0.25]
 [0.2  0.2  0.5  0.2  0.2 ]
 [0.65 0.65 0.2  0.5  0.2 ]
 [0.65 0.65 0.2  0.2  0.5 ]] 

Perfect Multiplicative Consistent IFPR (R_bar):
Row 1: (0.5000, 0.5000)	(0.6000, 0.2000)	(0.6923, 0.0588)	(0.7146, 0.0673)	(0.7218, 0.0704)
Row 2: (0.2000, 0.6000)	(0.5000, 0.5000)	(0.6000, 0.2000)	(0.6923, 0.0588)	(0.7146, 0.0673)
Row 3: (0.0588, 0.6923)	(0.2000, 0.6000)	(0.5000, 0.5000)	(0.6000, 0.2000)	(0.6923, 0.0588)
Row 4: (0.0673, 0.7146)	(0.0588, 0.6923)	(0.2000, 0.6000)	(0.5000, 0.5000)	(0.6000, 0.2000)
Row 5: (0.0704, 0.7218)	(0.0673, 0.7146)	(0.0588, 0.6923)	(0.2000, 0.6000)	(0.5000, 0.5000)


### 📏 Consistency Validation between Original and Perfect IFPR

This function computes the distance metric between the **original IFPR matrix** $R$ and the **perfectly consistent IFPR** $\bar{R}$:

$$
d(R, \bar{R}) = \frac{1}{(n - 1)(n - 2)} \sum_{i=1}^{n} \sum_{k=i+1}^{n} \left( |\bar{\mu}_{ik} - \mu_{ik}| + |\bar{\nu}_{ik} - \nu_{ik}| + |\bar{\pi}_{ik} - \pi_{ik}| \right)
$$

Where:
- $\mu, \nu$: Original membership and non-membership degrees  
- $\bar{\mu}, \bar{\nu}$: Adjusted values after enforcing consistency  
- $\pi = 1 - \mu - \nu$, and $\bar{\pi} = 1 - \bar{\mu} - \bar{\nu}$ (hesitation)

If $d < \tau$ (e.g., $\tau = 0.1$), the matrix is considered **sufficiently consistent** for applying IF-AHP scoring procedures.

✅ **Use this to validate** expert judgments before calculating weights or rankings.


In [74]:
def check_consistency(R, R_bar, tau=0.1):
    n = R.shape[0]
    total_diff = 0.0
    
    # Loop over the upper triangular indices (i < k)
    for i in range(n):
        for k in range(i+1, n):
            mu = R[i, k, 0]
            nu = R[i, k, 1]
            pi = 1 - mu - nu
            
            mu_bar = R_bar[i, k, 0]
            nu_bar = R_bar[i, k, 1]
            pi_bar = 1 - mu_bar - nu_bar
            
            diff = abs(mu_bar - mu) + abs(nu_bar - nu) + abs(pi_bar - pi)
            total_diff += diff
    
    # Use normalization factor as in the paper: (n-1)*(n-2) or 2*(n-1)(n-2) idk man
    denominator = (n - 1) * (n - 2)
    d = total_diff / denominator
    
    print("Distance d(R, R_bar) =", d)
    return d < tau

R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
    
# Check consistency:
consistent = check_consistency(R, R_bar, tau=0.1)
if consistent:
    print("The IFPR is consistent (d < 0.1).")
else:
    print("The IFPR is NOT consistent (d >= 0.1).")

Distance d(R, R_bar) = 0.16975383835175664
The IFPR is NOT consistent (d >= 0.1).


### 🔧 Algorithm 2: Repairing the IFPR Matrix for Consistency

This routine repairs the **Intuitionistic Fuzzy Preference Relation (IFPR)** matrix $R$ using **Algorithm 2**, a fusion-based iterative update process that aligns $R$ with a multiplicatively consistent matrix $\bar{R}$.

---

#### 📐 Update Formulas (Fusion Scheme)

For each off-diagonal element $(i, k)$, we update the membership and non-membership values using:

$$
\mu^{(p+1)}_{ik} = \frac{\mu^{(p)}_{ik}{}^{1 - \sigma} \cdot \bar{\mu}_{ik}{}^{\sigma}}{\mu^{(p)}_{ik}{}^{1 - \sigma} \cdot \bar{\mu}_{ik}{}^{\sigma} + (1 - \mu^{(p)}_{ik})^{1 - \sigma} \cdot (1 - \bar{\mu}_{ik})^{\sigma}}
$$

$$
\nu^{(p+1)}_{ik} = \frac{\nu^{(p)}_{ik}{}^{1 - \sigma} \cdot \bar{\nu}_{ik}{}^{\sigma}}{\nu^{(p)}_{ik}{}^{1 - \sigma} \cdot \bar{\nu}_{ik}{}^{\sigma} + (1 - \nu^{(p)}_{ik})^{1 - \sigma} \cdot (1 - \bar{\nu}_{ik})^{\sigma}}
$$

Where:
- $\sigma \in (0, 1)$ is the **fusion factor** (e.g., $\sigma = 0.8$)
- $\mu^{(p)}$ and $\nu^{(p)}$ are the values at iteration $p$
- $\bar{\mu}$ and $\bar{\nu}$ are the values from the perfectly consistent matrix $\bar{R}$

---

#### 🔁 Iterative Process

- Diagonal values are fixed to $(0.5, 0.5)$.
- Reciprocity: $R[k, i] = (\nu_{ik}, \mu_{ik})$ for all $i \neq k$.
- Stop iterating once consistency is achieved: $d(R^{(p+1)}, \bar{R}) < \tau$

---

📌 **Note:** If the matrix is already consistent initially, no repair is applied. Otherwise, the method converges in a few iterations for well-formed inputs.


In [76]:
def repair_ifahp_algorithm_2(R, sigma=0.5, tau=0.1, max_iter=100):
    # First, compute the perfect IFPR matrix.
    R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
    print("Initial consistency check:")
    if check_consistency(R, R_bar, tau):
        print("R is already consistent with R_bar; no repair needed.")
        return R

    # Start the iterative repair process.
    R_current = R.copy()
    n = R_current.shape[0]
    
    for p in range(max_iter):
        R_next = np.copy(R_current)
        
        # Update all off-diagonal entries using the ratio-based formulas.
        for i in range(n):
            for k in range(n):
                if i != k:
                    mu_p = R_current[i, k, 0]
                    nu_p = R_current[i, k, 1]
                    mu_bar = R_bar[i, k, 0]
                    nu_bar = R_bar[i, k, 1]
                    
                    # Membership update:
                    num_mu = (mu_p ** (1 - sigma)) * (mu_bar ** sigma)
                    den_mu = num_mu + ((1 - mu_p) ** (1 - sigma)) * ((1 - mu_bar) ** sigma)
                    mu_next = num_mu / den_mu if den_mu != 0 else 0.5
                    
                    # Non-membership update:
                    num_nu = (nu_p ** (1 - sigma)) * (nu_bar ** sigma)
                    den_nu = num_nu + ((1 - nu_p) ** (1 - sigma)) * ((1 - nu_bar) ** sigma)
                    nu_next = num_nu / den_nu if den_nu != 0 else 0.5
                    
                    R_next[i, k, 0] = mu_next
                    R_next[i, k, 1] = nu_next
        
        # Enforce the diagonal to remain (0.5, 0.5)
        for i in range(n):
            R_next[i, i] = [0.5, 0.5]
        
        # Enforce reciprocity for the lower-triangular part:
        for i in range(n):
            for k in range(i+1, n):
                mu_val, nu_val = R_next[i, k]
                R_next[k, i] = [nu_val, mu_val]
        
        print(f"Iteration {p+1} consistency check:")
        if check_consistency(R_next, R_bar, tau):
            print(f"Repair successful after {p+1} iterations.")
            return R_next
        
        R_current = R_next
    
    print("Max iterations reached; returning final repaired matrix.")
    return R_current

R_repaired = repair_ifahp_algorithm_2(R, sigma=0.8, tau=0.1, max_iter=100)
    
# --- Print the repaired IFPR matrix ---
print("\nRepaired IFPR matrix:")
for i in range(n):
    row_str = []
    for k in range(n):
        mu_val = R_repaired[i, k, 0]
        nu_val = R_repaired[i, k, 1]
        row_str.append(f"({mu_val:.4f}, {nu_val:.4f})")
    print("Row", i+1, ":", "\t".join(row_str))

Initial consistency check:
Distance d(R, R_bar) = 0.16975383835175664
Iteration 1 consistency check:
Distance d(R, R_bar) = 0.0204209644947799
Repair successful after 1 iterations.

Repaired IFPR matrix:
Row 1 : (0.5000, 0.5000)	(0.6000, 0.2000)	(0.6748, 0.0762)	(0.7022, 0.0893)	(0.7082, 0.0924)
Row 2 : (0.2000, 0.6000)	(0.5000, 0.5000)	(0.6000, 0.2000)	(0.6841, 0.0803)	(0.7022, 0.0893)
Row 3 : (0.0762, 0.6748)	(0.2000, 0.6000)	(0.5000, 0.5000)	(0.6000, 0.2000)	(0.6748, 0.0762)
Row 4 : (0.0893, 0.7022)	(0.0803, 0.6841)	(0.2000, 0.6000)	(0.5000, 0.5000)	(0.6000, 0.2000)
Row 5 : (0.0924, 0.7082)	(0.0893, 0.7022)	(0.0762, 0.6748)	(0.2000, 0.6000)	(0.5000, 0.5000)


### 🧮 Step 3: Computing Intuitionistic Fuzzy Weights of Criteria

This step calculates the **intuitionistic fuzzy weights** $\omega_i = (\mu_i, \nu_i)$ for each criterion $i$ based on the repaired IFPR matrix $R$.

---

#### 📌 Formula (26):

The weights are computed as follows:

$$
\mu_i = \frac{\sum_{k=1}^{n} \mu_{ik}}{\sum_{i=1}^{n} \sum_{k=1}^{n} (1 - \nu_{ik})}
$$

$$
\nu_i = 1 - \frac{\sum_{k=1}^{n} (1 - \nu_{ik})}{\sum_{i=1}^{n} \sum_{k=1}^{n} \mu_{ik}}
$$

Where:
- $\mu_{ik}$ and $\nu_{ik}$ are the membership and non-membership values in the IFPR matrix.
- $\omega_i$ is the intuitionistic fuzzy weight for criterion $i$.

---

#### ⚠️ Edge Case Handling:

If the denominator in either formula is near zero, the default fallback values $(\mu_i, \nu_i) = (0.5, 0.5)$ are used to avoid division by zero errors.

---

#### ✅ Output:

A list of $n$ weights, each represented as an ordered pair $(\mu_i, \nu_i)$.


In [78]:
def compute_criterion_weights(R):
    n = R.shape[0]
    weights = np.zeros((n, 2), dtype=float)
    
    # Calculate the denominators first (they are the same for all weights)
    sum_all_mu = 0.0
    sum_all_one_minus_nu = 0.0
    
    for i in range(n):
        for k in range(n):
            sum_all_mu += R[i, k, 0]  # Sum of all μ(i,k)
            sum_all_one_minus_nu += (1.0 - R[i, k, 1])  # Sum of all (1-ν(i,k))
    
    # Now calculate weights for each criterion
    for i in range(n):
        sum_mu_i = sum(R[i, k, 0] for k in range(n))  # Sum of μ(i,k) for row i
        sum_one_minus_nu_i = sum(1.0 - R[i, k, 1] for k in range(n))  # Sum of (1-ν(i,k)) for row i
        
        # Calculate weights according to formula 26
        if sum_all_one_minus_nu <= 1e-15 or sum_all_mu <= 1e-15:
            w_mu = 0.5
            w_nu = 0.5
        else:
            w_mu = sum_mu_i / sum_all_one_minus_nu
            w_nu = 1.0 - (sum_one_minus_nu_i / sum_all_mu)
        
        weights[i, 0] = w_mu
        weights[i, 1] = w_nu
    
    return weights

weights = compute_criterion_weights(R_repaired)

print("\nCriterion Weights (μ, ν):")
for i, (mu_w, nu_w) in enumerate(weights, start=1):
    print(f"ω_{i} = ({mu_w:.4f}, {nu_w:.4f})")


Criterion Weights (μ, ν):
ω_1 = (0.2174, 0.6095)
ω_2 = (0.1834, 0.6589)
ω_3 = (0.1400, 0.7151)
ω_4 = (0.1003, 0.7765)
ω_5 = (0.0654, 0.8247)



The intuitionistic fuzzy weights derived from our IF-AHP method are consistent with those presented in the referenced research paper. The values of the weights (each in the form of an intuitionistic fuzzy val)) are:

$$
\omega_1 = (0.2174, 0.6095)
$$
$$
\omega_2 = (0.1834, 0.6589)
$$
$$
\omega_3 = (0.1400, 0.7151)
$$
$$
\omega_4 = (0.1003, 0.7765)
$$
$$
\omega_5 = (0.0654, 0.8247)
$$

These match the fuzzy weights used in the case study section of the original paper. The following code block implements the method for **defuzzifying** these intuitionistic fuzzy values into **crisp weights** that can be used by ranking algorithms such as **VIKOR**, **TOPSIS**, etc.


### 🎯 STEP 4: Defuzzification of Intuitionistic Fuzzy Weights

This function converts the previously computed **intuitionistic fuzzy weights** into **crisp weights** using a combined approach based on:

- **Score function**:  
  $$
  S(\omega_i) = \mu_i - \nu_i
  $$

- **Accuracy function**:  
  $$
  H(\omega_i) = \mu_i + \nu_i
  $$

The score measures the net preference of each criterion, while the accuracy represents the total confidence in that preference. A combined score is calculated for each criterion as part of the defuzzification process.

The combined crisp value is then computed using:
$$
\text{Crisp Value}_i = 0.5 \cdot (S(\omega_i) + 1) \cdot (1 + H(\omega_i) - S(\omega_i))
$$

Finally, all crisp values are **normalized** to ensure they sum to 1:
$$
\text{Crisp Weight}_i = \frac{\text{Crisp Value}_i}{\sum_{j=1}^{n} \text{Crisp Value}_j}
$$

📌 **Note**: If the computed crisp values sum to zero (which is rare), equal weights are assigned to all criteria.


In [80]:
def convert_to_crisp_weights(weights):
    n = weights.shape[0]
    crisp_values = np.zeros(n)
    
    for i in range(n):
        mu = weights[i, 0]
        nu = weights[i, 1]
        
        # Score function: S(ω_i) = μ_i - ν_i
        score = mu - nu
        
        # Accuracy function: H(ω_i) = μ_i + ν_i
        accuracy = mu + nu
        
        # Combined score (this is one approach)
        crisp_values[i] = 0.5 * (score + 1) * (1 + accuracy - score)
    
    # Normalize to ensure sum is 1
    sum_crisp = np.sum(crisp_values)
    if sum_crisp > 0:
        crisp_weights = crisp_values / sum_crisp
    else:
        # Equal weights if all values are 0
        crisp_weights = np.ones(n) / n
        
    return crisp_weights

crisp_weights = convert_to_crisp_weights(weights)
print(crisp_weights)

[0.2665 0.2402 0.204  0.1633 0.126 ]
